In [1]:
import tensorflow as tf
import keras
from keras import Model
from keras import layers
import numpy as np

ModuleNotFoundError: No module named 'tensorflow'


#------------------------------------------------------------------------------------#
##  THIS IS A TRAINING MODEL, IMPLEMENTION DETAIL MAY DIFFER FROM PRODUCTION MODEL  ##
#------------------------------------------------------------------------------------#


# TODO
# THIS IS AN ABSTRACT BACKBONE SCRIPT, ADD ACTUAL IMPLEMENTATION LATER

# NOTE
# model detail

In [ ]:
# global variables
model = None # public

In [2]:
# custom layer definition
class Conv2Plus1D(layers.Layer):
    def __init__(self, kernel_size, filters = 1, strides = (1,1,1), padding = 'valid'):
        """kernel_size is depth width height"""
        super().__init__()
        wh_stride = (1, strides[1], strides[2])
        t_stride = (strides[0], 1, 1)
        self.seq = keras.Sequential([  
        # Spatial decomposition
        layers.Conv3D(filters=filters,
                      kernel_size=(1, kernel_size[1], kernel_size[2]), strides = wh_stride,
                      padding=padding),
        # Temporal decomposition
        layers.Conv3D(filters=filters, 
                      kernel_size=(kernel_size[0], 1, 1), strides = t_stride,
                      padding=padding)
        ])

    def call(self, x):
        return self.seq(x)

In [3]:
# hand model
class HandModel(Model):
    # run it every 2 frames in order to give it 2 temporal stride
    def __init__(self):
        super().__init__()
        self.conv21 = Conv2Plus1D(kernel_size = (25,3,3))
        self.ln = layers.LayerNormalization()

    def call(self, x, training= False):
        x = self.conv21(x)
        return self.ln(x, training= training)


In [ ]:
# pose model
class PoseModel(Model):
    def __init__(self, kernel_size = (17,3)):
        super().__init__()
        self.kernel_size = kernel_size
        self.dense_td = layers.TimeDistributed(layers.Dense(3, activation='relu'))
        self.conv2_td = layers.TimeDistributed(layers.Conv2D(filters=1, kernel_size = kernel_size))
        self.ln = layers.LayerNormalization()

    def call(self, input, training= False):
        # input shape:  batch time channel features
        # output shape: batch channel conv_slide
        x = self.dense_td(input, training = training)
        # time channel FC_result -> channel time FC_result
        x = layers.Permute((2, 1, 3))(x)
        # force add required "channel" input for 2dCNN (not to confuse with xyz channel)
        # shape to: xyz_channel, time, FC_result, 1
        x = tf.expand_dims(x, axis=4)
        x = self.conv2_td(x)
        x = tf.squeeze(x)
        #TODO ln axis
        return self.ln(x, training = training)
        

In [ ]:
# the main model class
class SLRModel(Model):
    def __init__(self):
        super().__init__()
        pass

    def call(self, inputs):
        # inputs 0: L, 1: R, 2: Pose
        
        pass


# the part that doesnt get executed when imported
if __name__ == "__main__":
    pass


def get_model():
    return model

def set_model(new_model):
    global model
    model = new_model

def train_or_fit():
    pass